<a href="https://colab.research.google.com/github/Stacy067/Deep-learning-and-Tensorflow/blob/main/TF_Certificate_Category_4_(%EC%8B%A4%EC%8A%B5)%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Category 4

RNN 을 활용한 텍스트 분류 (Text Classification)

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

NLP QUESTION

For this task you will build a classifier for the sarcasm dataset
The classifier should have a final layer with 1 neuron activated by sigmoid as shown.<br/><br/>
It will be tested against a number of sentences that the network hasn't previously seen<br/>
And you will be scored on whether sarcasm was correctly detected in those sentences


-------------------------------
**자연어 처리**<br/><br/>
이 작업에서는 sarcasm 데이터 세트에 대한 분류기를 작성합니다.
분류기는 1 개의 뉴런으로 이루어진 sigmoid 활성함수로 구성된 최종 층을 가져야합니다.<br/>
제출될 모델은 데이터셋이 없는 여러 문장에 대해 테스트됩니다.
그리고 당신은 그 문장에서 sarcasm 판별이 제대로 감지되었는지에 따라 점수를 받게 될 것입니다


-----------------------------------


# 실습

## STEP 1. import

**[코드]**

In [1]:
import json
import tensorflow as tf
import numpy as np
import urllib

# 필요한 모듈 import
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

## STEP 2. Load Dataset

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f0999eb9190>)

## STEP 3. Json 파일 로드

`datas` 변수에 `json`을 활용하여 로드

**[코드]**

In [3]:
with open('sarcasm.json') as f:
    datas = json.load(f)

잘 load 되었는지 체크합니다.

In [4]:
datas[:5]

[{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
  'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
  'is_sarcastic': 0},
 {'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
  'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
  'is_sarcastic': 0},
 {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
  'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
  'is_sarcastic': 1},
 {'article_link': 'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
  'headline': 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
  'is_sarcastic': 1},
 {'article_link': 'https://www.huffingtonpost.com/entry/jk-rowling-w

## STEP 4. 데이터셋 구성(sentences, labels)

* X (Feature): sentences
* Y (Label): label

빈 list를 생성합니다. (sentences, labels)

**[코드]**

In [5]:
sentences = []
labels = []

**[코드]**

In [6]:
for data in datas:
    # 이곳에 코드를 입력합니다.
    sentences.append(data['headline'])
    labels.append(data['is_sarcastic']) 

## STEP 5. Train / Validation Set 분리

20,000개를 기준으로 **데이터셋을 분리**합니다.


In [7]:
training_size = 20000

**[코드]**

In [8]:
train_sentences = sentences[:training_size]
train_labels = labels[:training_size]

**[코드]**

In [9]:
validation_sentences = sentences[training_size:]
validation_labels = labels[training_size:]

## STEP 6. Tokenizer 정의

단어의 토큰화를 진행합니다.

* `num_words`: 단어 max 사이즈를 지정합니다. 가장 **빈도수가 높은** 단어부터 저장합니다.
* `oov_token`: 단어 토큰에 없는 단어를 어떻게 표기할 것인지 지정해줍니다.

In [10]:
vocab_size = 1000
oov_tok = "<OOV>"

**[코드]**

In [11]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

## STEP 7. Tokenizer로 학습시킬 문장에 대한 토큰화 진행

`fit_on_texts`로 학습할 문장에 대하여 **토큰화**를 진행합니다.

**[코드]**

In [12]:
tokenizer.fit_on_texts(train_sentences)

## STEP 8. 문장(sentences)을 토큰으로 변경 (치환)

* `texts_to_sequences`: 문장을 숫자로 **치환** 합니다. 
* Train Set, Valid Set 모두 별도로 적용해주어야 합니다.

**[코드]**

In [13]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

## STEP 9. 시퀀스의 길이를 맞추기

3가지 옵션을 입력해 줍니다.

* `maxlen`: 최대 문장 길이를 정의합니다. 최대 문장길이보다 길면, 잘라냅니다.
* `truncating`: 문장의 길이가 `maxlen`보다 길 때 앞을 자를지 뒤를 자를지 정의합니다.
* `padding`: 문장의 길이가 `maxlen`보다 짧을 때 채워줄 값을 앞을 채울지, 뒤를 채울지 정의합니다.

In [14]:
# 한 문장의 최대 단어 숫자
max_length = 120

# 잘라낼 문장의 위치
trunc_type='post'

# 채워줄 문장의 위치
padding_type='post'

**[코드]**

In [15]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

In [16]:
train_padded.shape, validation_padded.shape

((20000, 120), (6709, 120))

## Step 10. label 값을 numpy array로 변환

model이 `list` type은 받아들이지 못하므로, numpy array로 변환합니다.

**[코드]**

In [17]:
train_labels = np.array(train_labels)
validation_labels = np.array(validation_labels)

In [18]:
train_labels.shape, validation_labels.shape

((20000,), (6709,))

-------------------------------------------------------------


## STEP 11. 모델 정의 (Sequential)

In [19]:
embedding_dim = 16

**[코드]**

In [20]:
model = Sequential([
    # 이곳에 모델링을 진행합니다.
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    # 마지막 출력층은 뉴런(노드)의 갯수를 1로 지정합니다.
    Dense(1, activation='sigmoid')
])

요약 확인

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16000     
                                                                 
 bidirectional (Bidirectiona  (None, 120, 128)         41472     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 1

## STEP 12. 컴파일 (compile)

1. `optimizer`는 가장 최적화가 잘되는 알고리즘인 'adam'을 사용합니다.
2. `loss`는 이진 분류이기 때문에 `binary_crossentropy`를 사용합니다.

**[코드]**

In [22]:
model.compile(# 이곳에 코드를 입력합니다.
              optimizer='adam', loss='binary_crossentropy', metrics=['acc']
              )

## STEP 13. ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

**[코드]**

In [23]:
checkpoint_path = 'shkim_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

## STEP 14. 학습 (fit)

In [24]:
epochs=10

**model.fit()**

**[코드]**

In [25]:
# 이곳에 코드를 입력합니다.
history = model.fit(train_padded, train_labels,
                    validation_data=(validation_padded, validation_labels),
                    callbacks=[checkpoint],
                    epochs=epochs)

Epoch 1/10
624/625 [============================>.] - ETA: 0s - loss: 0.4635 - acc: 0.7702
Epoch 1: val_loss improved from inf to 0.39028, saving model to shkim_checkpoint.ckpt
625/625 [==============================] - 25s 25ms/step - loss: 0.4633 - acc: 0.7703 - val_loss: 0.3903 - val_acc: 0.8213
Epoch 2/10
623/625 [============================>.] - ETA: 0s - loss: 0.3579 - acc: 0.8370
Epoch 2: val_loss improved from 0.39028 to 0.37729, saving model to shkim_checkpoint.ckpt
625/625 [==============================] - 14s 22ms/step - loss: 0.3576 - acc: 0.8371 - val_loss: 0.3773 - val_acc: 0.8261
Epoch 3/10
624/625 [============================>.] - ETA: 0s - loss: 0.3303 - acc: 0.8535
Epoch 3: val_loss improved from 0.37729 to 0.37708, saving model to shkim_checkpoint.ckpt
625/625 [==============================] - 14s 22ms/step - loss: 0.3301 - acc: 0.8537 - val_loss: 0.3771 - val_acc: 0.8204
Epoch 4/10
625/625 [==============================] - ETA: 0s - loss: 0.3147 - acc: 0.8613
E

## STEP 15. 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

**[코드]**

In [26]:
# 이곳에 코드를 입력합니다.
model.load_weights(checkpoint_path)